# MNIST Digits Recognition

In [ ]:
# ######################################### E2E Demo example precondition ##############################################

# # You need to generate secret to have ability read datasource files from S3 bucket by Spark application (Airflow DAG)
# # Copy template "object_store_secret.yaml.tpl" from shared/ezua-tutorials/current-release/Data-Analytics/Spark directory to e.g. user folder

# # Users have to pull the following images manually and make available in local repo for clusters in airgap network to run the example.
  # "docker.io/kubeflowkatib/kubeflow-pipelines-launcher", "nikenano/launchernew:latest", "quay.io/aipipeline/kserve-component:v0.10.1"

import kfp

kfp_client = kfp.Client()
namespace = kfp_client.get_user_namespace()

!sed -e "s/\$AUTH_TOKEN/$AUTH_TOKEN/" /mnt/user/object_store_secret.yaml.tpl > /tmp/object_store_secret.yaml
!kubectl apply -f /tmp/object_store_secret.yaml -n {namespace}

In [ ]:
%%bash
# # Local airgap repo path prefix shall be added to images mentioned above in following yaml files.
  # "component/katib-launcher-component.yaml", "component/kubeflow-launcher-component.yaml", "component/kserve-component.yaml"

if [[ -n "$AIRGAP_REGISTRY" ]]; then
    echo 'AIRGAP_REGISTRY is set - updating component YAML files'
    for file in `ls component/*.yaml`; do
        echo "$file";
        sed "\%image: *$AIRGAP_REGISTRY%b; s%image: *%image: $AIRGAP_REGISTRY%g" --in-place "$file";
    done;
fi

In [ ]:
import os
# Specify proxy and no_proxy values for pipeline from notebook environment variables. Update manually if needed.
http_proxy = os.environ["http_proxy"] ## "http://<proxy_host>:<port>"
# After default.svc append comma separated list of IP address, hostname, CIDR block, service name, etc.
no_proxy = os.environ["no_proxy"] ## "localhost,.cluster.local,.svc,.default.svc, <host>:<port>, x.x.x.x/y,..."
# Specify airgap registry to use as a prefix for images
airgap_registry = os.environ["AIRGAP_REGISTRY"]

In [ ]:
# Specify the name of the user mounted directory
user_mounted_dir_name = "user"

In [ ]:
import os
import uuid
import kfp
import kfp.dsl as dsl
from kfp import components
from pathlib import Path

######################################### Storage Parameters ##############################################

# The path that is used inside the pods to mount training data
mnt_path = "/mnt/data/"
uuid = uuid.uuid4().hex[:4]

# The initial training data is written to the user volume by the spark job in the Apache Parquet format.
initial_training_data_dir = f"{str(Path.home())}/{user_mounted_dir_name}/mnist-spark-data"

# The path is relative, final_training_data_dir should be in the same folder with the notebook
final_training_data_dir=f"training-{uuid}"

os.mkdir("/mnt/user/" + final_training_data_dir, 0o777)

######################################## KFP parameters ##################################################

# Name of the Katib experiment
name = f"mnist-experiment-{uuid}"
print(f"Katib experiment name: {name}")

# Number of epoch to train the model
training_steps="100"
volume_str = f"model-volume-{uuid}"

# Create volume to train and serve the model.
!sed -e "s/pvc-name/$volume_str/" ./pvc.yaml > /tmp/pvc.yaml
!kubectl apply -f /tmp/pvc.yaml

kfp_client = kfp.Client()
namespace = kfp_client.get_user_namespace()

In [ ]:
# Transformation of the training data from Apache Parquet format to the format that is required for the MNIST example
import pandas as pd
import os

if not os.path.exists(final_training_data_dir):
    os.makedirs(final_training_data_dir)
    
with open(final_training_data_dir + "/train-images-idx3-ubyte.gz", 'wb') as f1, \
     open(final_training_data_dir + "/t10k-images-idx3-ubyte.gz", 'wb') as f2, \
     open(final_training_data_dir + "/train-labels-idx1-ubyte.gz", 'wb') as f3, \
     open(final_training_data_dir + "/t10k-labels-idx1-ubyte.gz", 'wb') as f4:
         mnist_parquet = pd.read_parquet(initial_training_data_dir)
         x_train, x_test, y_train, y_test = mnist_parquet["content"]
         f1.write(x_train)
         f2.write(x_test)
         f3.write(y_train)
         f4.write(y_test)

In [ ]:
env = []
if http_proxy:
    env.append({"name": "HTTP_PROXY", "value":http_proxy})
    env.append({"name": "HTTPS_PROXY", "value":http_proxy})
    env.append({"name": "http_proxy", "value":http_proxy})
    env.append({"name": "https_proxy", "value":http_proxy})
if no_proxy:
    env.append({"name": "no_proxy", "value":no_proxy})
    env.append({"name": "NO_PROXY", "value":no_proxy})

In [ ]:
# You should define number of training steps in the arguments.
def create_katib_experiment_task(experiment_name, experiment_namespace, training_steps):
    # Trial count specification.
    max_trial_count = 5
    max_failed_trial_count = 3
    parallel_trial_count = 2

    # Objective specification.
    objective = {
        "objectiveMetricName": "loss",
        "type": "minimize",
        "goal": 0.001,
    }

    # Algorithm specification.
    algorithm = {
        "algorithmName": "random",
    }

    # Experiment search space.
    # In this example we tune learning rate and batch size.
    parameters = [
        {
            "name": "learning_rate",
            "parameterType": "double",
            "feasibleSpace": {
                "min": "0.01",
                "max": "0.05",
            },
        },
        {
            "name": "batch_size",
            "parameterType": "int",
            "feasibleSpace": {
                "min": "80",
                "max": "100",
            },
        },
    ]

    # Experiment Trial template.
    trial_spec = {
        "apiVersion": "kubeflow.org/v1",
        "kind": "TFJob",
        "spec": {
            "tfReplicaSpecs": {
                "Chief": {
                    "replicas": 1,
                    "restartPolicy": "OnFailure",
                    "template": {
                        "metadata": {
                            "annotations": {
                                "sidecar.istio.io/inject": "false"
                            }
                        },
                        "spec": {
                            "containers": [
                                {
                                    "name": "tensorflow",
                                    "image": f"{airgap_registry}gcr.io/mapr-252711/kubeflow/kfexamples/docker.io/liuhougangxa/tf-estimator-mnist",
                                    "command": [
                                        "python",
                                        "/opt/model.py",
                                        f"--tf-data-dir={mnt_path}{final_training_data_dir}/",
                                        f"--tf-train-steps={str(training_steps)}",
                                        f"--tf-export-dir={mnt_path}{final_training_data_dir}/",
                                        "--tf-learning-rate=${trialParameters.learningRate}",
                                        "--tf-batch-size=${trialParameters.batchSize}"
                                    ],
                                    "env": env,
                                    "volumeMounts": [
                                        {
                                            "mountPath": mnt_path,
                                            "name": "data-volume"
                                        }
                                    ],
                                    "resources":
                                    {
                                        "limits":{
                                            "cpu":"2",
                                            "memory":"1Gi"
                                        },
                                        "requests":{
                                            "cpu":"100m",
                                            "memory":"50Mi"
                                        }
                                    }      
                                }
                            ],
                            "imagePullSecrets": [
                                {
                                    "name": "hpe-imagepull-secrets"
                                }
                            ],
                            "volumes": [
                                {
                                    "name": "data-volume",
                                    "persistentVolumeClaim": {
                                        "claimName": "user-pvc"
                                    }
                                }
                            ]
                        }
                    }
                },
                "Worker": {
                    "replicas": 1,
                    "restartPolicy": "OnFailure",
                    "template": {
                        "metadata": {
                            "annotations": {
                                "sidecar.istio.io/inject": "false"
                            }
                        },
                        "spec": {
                            "containers": [
                                {
                                    "name": "tensorflow",
                                    "image": f"{airgap_registry}gcr.io/mapr-252711/kubeflow/kfexamples/docker.io/liuhougangxa/tf-estimator-mnist",
                                    "command": [
                                        "python",
                                        "/opt/model.py",
                                        f"--tf-data-dir={mnt_path}{final_training_data_dir}/",
                                        f"--tf-train-steps={str(training_steps)}",
                                        f"--tf-export-dir={mnt_path}{final_training_data_dir}/",
                                        "--tf-learning-rate=${trialParameters.learningRate}",
                                        "--tf-batch-size=${trialParameters.batchSize}"
                                    ],
                                    "env": env,
                                    "volumeMounts": [
                                        {
                                            "mountPath": mnt_path,
                                            "name": "data-volume"
                                        }
                                    ],
                                    "resources":
                                    {
                                        "limits":{
                                            "cpu":"2",
                                            "memory":"1Gi"
                                        },
                                        "requests":{
                                            "cpu":"100m",
                                            "memory":"50Mi"
                                        }
                                    }
                                }
                            ],
                            "imagePullSecrets": [
                                {
                                    "name": "hpe-imagepull-secrets"
                                }
                            ],
                            "volumes": [
                                {
                                    "name": "data-volume",
                                    "persistentVolumeClaim": {
                                        "claimName": "user-pvc"
                                    }
                                }
                            ]
                        }
                    }
                }
            }
        }
    }

    # Configure parameters for the Trial template.
    trial_template = {
        "primaryContainerName": "tensorflow",
        "trialParameters": [
            {
                "name": "learningRate",
                "description": "Learning rate for the training model",
                "reference": "learning_rate",
            },
            {
                "name": "batchSize",
                "description": "Batch size for the model",
                "reference": "batch_size",
            },
        ],
        "trialSpec": trial_spec,
    }

    # Create an Experiment from the above parameters.
    experiment_spec = {
        "maxTrialCount": max_trial_count,
        "maxFailedTrialCount": max_failed_trial_count,
        "parallelTrialCount": parallel_trial_count,
        "objective": objective,
        "algorithm": algorithm,
        "parameters": parameters,
        "trialTemplate": trial_template,
    }

    # Create the KFP task for the Katib Experiment.
    # Experiment Spec should be serialized to a valid Kubernetes object.
    katib_experiment_launcher_op = components.load_component_from_file("component/katib-launcher-component.yaml")

    op = katib_experiment_launcher_op(
        experiment_name=experiment_name,
        experiment_namespace=experiment_namespace,
        experiment_spec=experiment_spec,
        experiment_timeout_minutes=60,
        delete_finished_experiment=False)

    return op

In [ ]:
# This function converts Katib Experiment HP results to args.

from kfp import components
@dsl.component(base_image = f"{airgap_registry}python:3.7")
def convert_katib_results(katib_results: dict) -> str:
    import json
    import pprint
    katib_results_json = katib_results
    print("Katib results:")
    pprint.pprint(katib_results_json)
    best_hps = []
    for pa in katib_results_json["currentOptimalTrial"]["parameterAssignments"]:
        if pa["name"] == "learning_rate":
            best_hps.append("--tf-learning-rate=" + pa["value"])
        elif pa["name"] == "batch_size":
            best_hps.append("--tf-batch-size=" + pa["value"])
    print(f"Best Hyperparameters: {best_hps}")
    return " ".join(best_hps)

# You should define the TFJob name, namespace, number of training steps, output of Katib and model volume tasks in the arguments.
def create_tfjob_task(tfjob_name, tfjob_namespace, training_steps, katib_op, model_volume_op, http_proxy, no_proxy):
    import json
    # Get parameters from the Katib Experiment.
    # Parameters are in the format "--tf-learning-rate=0.01 --tf-batch-size=100"
    best_hp_op = convert_katib_results(katib_results = katib_op.output)
    best_hp_op.set_caching_options(False)
    # Setting the proxy and no_proxy to allow download of the required packages inside the component.
    if no_proxy:
        no_proxy += ',.kubeflow'
        best_hp_op.set_env_variable('no_proxy', no_proxy)
        best_hp_op.set_env_variable('NO_PROXY', no_proxy)
    if http_proxy : 
        best_hp_op.set_env_variable('http_proxy', http_proxy)
        best_hp_op.set_env_variable('https_proxy', http_proxy)
        best_hp_op.set_env_variable('HTTP_PROXY', http_proxy)
        best_hp_op.set_env_variable('HTTPS_PROXY', http_proxy)
    
    best_hps = str(best_hp_op.output)

    # Create the TFJob Chief and Worker specification with the best Hyperparameters.
    tfjob_chief_spec = {
        "replicas": 1,
        "restartPolicy": "OnFailure",
        "template": {
            "metadata": {
                "annotations": {
                    "sidecar.istio.io/inject": "false"
                }
            },
            "spec": {
                "containers": [
                    {
                        "name": "tensorflow",
                        "image": f"{airgap_registry}gcr.io/mapr-252711/kubeflow/kfexamples/docker.io/liuhougangxa/tf-estimator-mnist",
                        "command": [
                            "sh",
                            "-c"
                        ],
                        "args": [
                            f"python /opt/model.py --tf-data-dir={mnt_path}{final_training_data_dir}/ --tf-export-dir=/mnt/export --tf-train-steps={training_steps} {best_hps}"
                        ],
                        "env": env,
                        "volumeMounts": [
                            {
                                "mountPath": "/mnt/export",
                                "name": "model-volume"
                            },
                            {
                                "mountPath": mnt_path,
                                "name": "data-volume"
                            }
                        ],
                        "resources":
                        {
                            "limits":{
                                "cpu":"2",
                                "memory": "1Gi"
                            },
                            "requests":{
                                "cpu": "100m",
                                "memory":"50Mi"
                            }
                        }
                    }
                ],
                "imagePullSecrets": [
                    {
                        "name": "hpe-imagepull-secrets"
                    }
                ],
                "volumes": [
                    {
                        "name": "model-volume",
                        "persistentVolumeClaim": {
                            "claimName": volume_str
                        }
                    },
                    {
                        "name": "data-volume",
                        "persistentVolumeClaim": {
                            "claimName": "user-pvc"
                        }
                    }
                ]
            }
        }
    }

    tfjob_worker_spec = {
        "replicas": 1,
        "restartPolicy": "OnFailure",
        "template": {
            "metadata": {
                "annotations": {
                    "sidecar.istio.io/inject": "false"
                }
            },
            "spec": {
                "containers": [
                    {
                        "name": "tensorflow",
                        "image": f"{airgap_registry}gcr.io/mapr-252711/kubeflow/kfexamples/docker.io/liuhougangxa/tf-estimator-mnist",
                        "command": [
                            "sh",
                            "-c",
                        ],
                        "args": [
                          f"python /opt/model.py --tf-data-dir={mnt_path}{final_training_data_dir}/ --tf-export-dir=/mnt/export --tf-train-steps={training_steps} {best_hps}" 
                        ],
                        "env": env,
                        "volumeMounts": [
                            {
                                "mountPath": "/mnt/export",
                                "name": "model-volume"
                            },
                            {
                                "mountPath": mnt_path,
                                "name": "data-volume"
                            }
                        ],
                        "resources":
                        {
                            "limits":{
                                "cpu":"2",
                                "memory":"1Gi"
                            },
                            "requests":{
                                "cpu": "100m",
                                "memory":"50Mi"
                            }
                        }
                    }
                ],
                "imagePullSecrets": [
                    {
                        "name": "hpe-imagepull-secrets"
                    }
                ],
                "volumes": [
                    {
                        "name": "model-volume",
                        "persistentVolumeClaim": {
                            "claimName": volume_str
                        }
                    },
                    {
                        "name": "data-volume",
                        "persistentVolumeClaim": {
                            "claimName": "user-pvc"
                        }
                    }
                ]
            }
        }
    }

    # Create the KFP task for the TFJob.
    tfjob_launcher_op = components.load_component_from_file("component/kubeflow-launcher-component.yaml")
    
    op = tfjob_launcher_op(
        name=tfjob_name,
        namespace=tfjob_namespace,
        chief_spec=json.dumps(tfjob_chief_spec),
        worker_spec=json.dumps(tfjob_worker_spec),
        tfjob_timeout_minutes=60,
        delete_finished_tfjob=False)
    return op

In [ ]:
def create_serving_task(model_name, model_namespace, tfjob_op, model_volume_op):
    api_version = 'serving.kserve.io/v1beta1'
    serving_component_url = 'component/kserve-component.yaml'

    # Uncomment the following two lines if you are using KFServing v0.6.x or v0.5.x
    # api_version = 'serving.kubeflow.org/v1beta1'
    # serving_component_url = 'component/kfserving-component.yaml'

    inference_service = f'''
      apiVersion: "{api_version}"
      kind: "InferenceService"
      metadata:
        name: {name}
        namespace: {namespace}
        annotations:
          "sidecar.istio.io/inject": "false"
      spec:
        predictor:
          tensorflow:
            storageUri: "pvc://{volume_str}/"
            resources:
              limits:
                cpu: '2'
                memory: 1Gi
              requests:
                cpu: 100m
                memory: 50Mi
      '''

    serving_launcher_op = components.load_component_from_file(serving_component_url)
    serving_launcher_op(action="apply", inferenceservice_yaml=inference_service).after(tfjob_op)

In [ ]:
@dsl.pipeline(
    name=name,
    description="An end to end mnist example including hyperparameter tuning, train and inference"
)

def mnist_pipeline(name: str, namespace: str):

    # Run the hyperparameter tuning with Katib.
    katib_op = create_katib_experiment_task(name, namespace, training_steps)

    model_volume_op =  volume_str

    # Run the distributive training with TFJob.
    tfjob_op = create_tfjob_task(name, namespace, training_steps, katib_op, model_volume_op, http_proxy, no_proxy)

    # Create the KServe inference.
    create_serving_task(name, namespace, tfjob_op, model_volume_op)
    print("Volume: ", volume_str)

# Run the Kubeflow Pipeline in the user's namespace.
run_id = kfp_client.create_run_from_pipeline_func(pipeline_func=mnist_pipeline, namespace=namespace, 
                                                  arguments={"name": name,
                                                             "namespace" : namespace}).run_id
print(f"Run ID: {run_id}")

kfp_client.wait_for_run_completion(run_id=run_id, timeout=36000)

In [ ]:
import numpy as np
from PIL import Image
import requests

# Pipeline Run should be succeeded.
kfp_run = kfp_client.get_run(run_id=run_id)
if kfp_run.state == "SUCCEEDED":
    print(f"Run {run_id} has been Succeeded\n")

    # Specify the image URL here.
    image = Image.open("image/9.bmp")
    data = np.array(image.convert('L').resize((28, 28))).astype(np.float64).reshape(-1, 28, 28, 1)
    data_formatted = np.array2string(data, separator=",", formatter={"float": lambda x: "%.1f" % x})
    json_request = f'{{ "instances" : {data_formatted} }}'

    # Specify the prediction URL. If you are runing this notebook outside of Kubernetes cluster, you should set the Cluster IP.
    url = f"http://{name}-predictor.{namespace}.svc.cluster.local/v1/models/{name}:predict"
    response = requests.post(url, data=json_request)

    print("Prediction for the image")
    display(image)
    print(response.json())